In [ ]:
import findspark
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import json
import csv
import os
import uuid
import sys

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [ ]:
#Initialize spark session
findspark.init()
SparkContext.setSystemProperty('spark.executor.memory', '4g')
conf = SparkConf().setAppName("combinetweets").setMaster("local")
sc = SparkContext(conf=conf)

In [ ]:
#Get list of paths for all files saved by scrapper

# basepath = r"C:\Users\Vineet\Desktop\cs226\snscrapetest\data"
basepath = r"D:\Vineet\PROJECTS\sns\snscrapetest\14gbdata"

foldername = basepath
pathlist = []

for root, dirs, files in os.walk(foldername):
    for file in files:
        if (file.endswith(".json")):
            pathlist.append(os.path.join(root, file))

In [ ]:
#Helper functions

#function to filter required fields from raw data
def get_data(rawdata):
    data = {}
    data['url'] = rawdata['url']
    data['date'] = rawdata['date']
    data['content'] = rawdata['content']
    data['tweetid'] = rawdata['id']
    data['user_username'] = rawdata['user']['username']
    data['user_id'] = rawdata['user']['id']
    data['user_displayname'] = rawdata['user']['displayname']
    data['user_followersCount'] = rawdata['user']['followersCount']
    data['user_friendsCount'] = rawdata['user']['friendsCount']
    data['user_location'] = rawdata['user']['location']
    data['user_url'] = rawdata['user']['url']
    data['replyCount'] = rawdata['replyCount']
    data['retweetCount'] = rawdata['retweetCount']
    data['likeCount'] = rawdata['likeCount']
    data['lang'] = rawdata['lang']
    return data


#function to save combined tweet json
def savetweets(tweets):
    fn = 'data-analytics-tweet-wc'.format(uuid.uuid4().hex)
    fpath = 'D:\\Vineet\\PROJECTS\\sns\\snscrapetest\\combined\\{}.json'.format(fn)
    temp = open(fpath, 'w')
    json.dump(tweets,temp)
    temp.close()
    print(fn)

In [ ]:
%%time

#Combining all tweets from all files
total_files = 0
total_records = 0
reset_limit = 5000001
reset_counter = 0

tweets = []
count = 0

for p in pathlist:
    print(p)
    textFile = sc.textFile(p)
    total_files +=1
    rawdata = textFile.collect()
    total_records += len(rawdata)
    
    for d in rawdata:
        if (reset_counter < reset_limit):
            json_object = json.loads(d)
            
            # Considering only english tweets
            if (json_object['lang'] == 'en'):
                payload = get_data(json_object)
                tweets.append(payload)
                reset_counter += 1
        else:
            savetweets(tweets)
            print("saved---------")
            tweets = []
            reset_counter = 0
            
savetweets(tweets)
print('Total File Count {} Total tweet Count {}'.format(total_files, total_records))
print('Completed---------')